In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import TFAutoModel, AutoTokenizer , TFBertModel
from sklearn.utils import resample

device = tf.config.experimental.list_physical_devices('GPU')
if device:
    tf.config.experimental.set_memory_growth(device[0], True)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [2]:
data_set = pd.read_csv('spam.csv')
data_set['spam'] = data_set['Category'].apply(lambda x : 1 if x == 'spam' else 0 )

spam = data_set[data_set['Category'] == 'spam'] 
ham = data_set[data_set['Category'] == 'ham']

resampled_ham = resample(ham , 
                          replace = True , 
                          n_samples = len(spam) ,
                          random_state = 42)

resampled_data = pd.concat([resampled_ham , spam])

text = resampled_data['Message'].tolist()
labels = resampled_data['spam'].values

In [3]:
X_train, X_test, y_train, y_test = train_test_split(text, labels, stratify=labels, test_size=0.2, random_state=42)

train_output = tokenizer(X_train, padding=True, truncation=True, max_length=128, return_tensors='tf')
test_output = tokenizer(X_test, padding=True, truncation=True, max_length=128, return_tensors='tf')

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_output), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_output), y_test))

batch_size = 32
train_dataset = train_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [4]:
import tensorflow as tf
from transformers import BertTokenizerFast, TFBertModel

class SpamClassifier(tf.keras.Model):
    def __init__(self, **kwargs):
        super(SpamClassifier, self).__init__(**kwargs)
        self.bert = TFBertModel.from_pretrained('bert-base-uncased')
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        bert_output = self.bert(inputs)[1]
        return self.dense(bert_output)

# Instantiate and compile the model
model = SpamClassifier()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Save the model architecture as a JSON string
model_json = model.to_json()
with open("Sentiment_analysis.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights
model.save_weights("Sentiment_analysis_weights.h5")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

X_train_tokens = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='tf')
X_test_tokens = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='tf')


X_train_input_ids = X_train_tokens['input_ids']
X_test_input_ids = X_test_tokens['input_ids']

loss, accuracy = model.evaluate(X_test_input_ids, y_test, verbose=2)

print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Overall Loss: {loss:.2f}')





10/10 - 67s - loss: 0.7074 - accuracy: 0.5017 - 67s/epoch - 7s/step
Accuracy: 50.17%
Overall Loss: 0.71


In [6]:
model_json = model.to_json()
with open("Sentiment_analysis.json", "w") as json_file:
    json_file.write(model_json)


model.save_weights("Sentiment_analysis_weights.h5")
model.summary()

Model: "spam_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_bert_model (TFBertModel  multiple                  109482240 
 )                                                               
                                                                 
 dense (Dense)               multiple                  769       
                                                                 
Total params: 109483009 (417.64 MB)
Trainable params: 109483009 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
